In [ ]:
## AGENDA
## - Page Rank

In [ ]:
#from py2neo import Graph
#graph = Graph("bolt://neo4j.sktai.io:30074", auth=("neo4j","!neo4j00"))

In [ ]:
# https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/tag/3.5.0.1
#mv ./apoc-3.5.0.1-all.jar $Neo4J_HOME/plugins

In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [2]:
#gph_conn = Neo4jConnection(uri="bolt://neo4j.sktai.io:30074", user="neo4j", pwd="!neo4j00")
#gph_conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="!neo4j00")
gph_conn = Neo4jConnection(uri="bolt://127.0.0.1:7687", user="neo4j", pwd="!neo4j00")

In [3]:
gph_conn.query("""
               CALL db.schema();
               """)

[<Record nodes=[<Node id=-2 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>] relationships=[<Relationship id=-2 nodes=(<Node id=-2 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>, <Node id=-2 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>) type='LINKED_TO' properties={}>]>]

In [4]:
gph_conn.query("""
          MATCH (n)
          DETACH DELETE n
          """)

[]

In [ ]:
## SET DATA

In [5]:
G = { 
    'A': {'B': 1, 'D': 1}, 
    'B': {'A': 1}, 
    'C': {'B': 1}, 
    'D': {'B': 1}, 
}

In [ ]:
## Page Rank from scratch

In [11]:
def page_rank(G, d=0.85, tolerance=0.01, max_iterations=50):
    """Returns the PageRank of the nodes in the graph.
    :param dict G: the graph
    :param float d: the damping factor
    :param flat tol: tolerance to determine algorithm convergence
    :param int max_iter: max number of iterations
    """
    N = len(G)
    pr = dict.fromkeys(G, 1.0)
    print("======= Initialization")
    print(pr)
    outgoing_degree = {k: len(v) for k, v in G.items()}

    # main loop
    for it in range(max_iterations):
        print("======= Iteration", it)
        old_pr = dict(pr)
        pr = dict.fromkeys(old_pr.keys(), 0)
        for node in G:
            for neighbor in G[node]:
                pr[neighbor] += d * old_pr[node] / outgoing_degree[node]
            pr[node] += (1 - d)
        print(pr)
        # check convergence
        mean_diff_to_prev_pr = sum([abs(pr[n] - old_pr[n]) for n in G])/N
        if mean_diff_to_prev_pr < tolerance:
            return pr
    raise Exception(
        f'PageRank failed after max iteration = {max_iterations}'
        f' (err={mean_diff_to_prev_pr} > tol = {tolerance})'
    )

In [12]:
pr = page_rank(G)

======= Initialization
{'A': 1.0, 'B': 1.0, 'C': 1.0, 'D': 1.0}
======= Iteration 0
{'A': 1.0, 'B': 2.275, 'C': 0.15000000000000002, 'D': 0.575}
======= Iteration 1
{'A': 2.0837499999999998, 'B': 1.19125, 'C': 0.15000000000000002, 'D': 0.575}
======= Iteration 2
{'A': 1.1625625, 'B': 1.6518437499999998, 'C': 0.15000000000000002, 'D': 1.0355937499999999}
======= Iteration 3
{'A': 1.5540671874999998, 'B': 1.6518437499999998, 'C': 0.15000000000000002, 'D': 0.6440890625}
======= Iteration 4
{'A': 1.5540671874999998, 'B': 1.4854542578124998, 'C': 0.15000000000000002, 'D': 0.8104785546875}
======= Iteration 5
{'A': 1.412636119140625, 'B': 1.626885326171875, 'C': 0.15000000000000002, 'D': 0.8104785546875}
======= Iteration 6
{'A': 1.5328525272460936, 'B': 1.5667771221191407, 'C': 0.15000000000000002, 'D': 0.7503703506347656}
======= Iteration 7
{'A': 1.4817605538012697, 'B': 1.5667771221191404, 'C': 0.15000000000000002, 'D': 0.8014623240795897}
======= Iteration 8
{'A': 1.4817605538012693, 'B

In [ ]:
## via GDS

In [15]:
gph_conn.query("""
               CREATE (A:Node {name: "A"})
               CREATE (B:Node {name: "B"})
               CREATE (C:Node {name: "C"})
               CREATE (D:Node {name: "D"})
               
               CREATE (A)-[:LINKED_TO {weight: 1}]->(B)
               CREATE (A)-[:LINKED_TO {weight: 1}]->(D)
               CREATE (B)-[:LINKED_TO {weight: 1}]->(A)
               CREATE (C)-[:LINKED_TO {weight: 1}]->(B)
               CREATE (D)-[:LINKED_TO {weight: 1}]->(B)
               """)

[]

In [16]:
gph_conn.query("""
               CALL gds.graph.drop("graph")
               """)
gph_conn.query("""
               CALL gds.graph.create("graph", "Node", "LINKED_TO")
               """)

[<Record graphName='graph' nodeProjection={'Node': {'properties': {}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {}}} nodeCount=4 relationshipCount=5 createMillis=4>]

In [17]:
gph_conn.query("""
               CALL gds.pageRank.stream("graph", {})
               YIELD nodeId, score
               RETURN gds.util.asNode(nodeId).name AS nodeName, score
               ORDER BY score DESC
               """)

[<Record nodeName='B' score=1.5238865334540603>,
 <Record nodeName='A' score=1.4373937157914038>,
 <Record nodeName='D' score=0.7569374122889712>,
 <Record nodeName='C' score=0.15000000000000002>]

In [18]:
gph_conn.query("""
               CALL gds.alpha.eigenvector.stream("graph", {})
               YIELD nodeId, score
               RETURN gds.util.asNode(nodeId).name AS nodeName, score
               ORDER BY score DESC
               """)

[<Record nodeName='B' score=18.965116918087006>,
 <Record nodeName='A' score=14.299190640449524>,
 <Record nodeName='D' score=10.795747071504593>,
 <Record nodeName='C' score=0.25>]